## Imports

In [1]:
import os
from pathlib import Path

import numpy as np

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "4"
import tensorflow as tf

tf.get_logger().setLevel("ERROR")


### Extract the weights for all 25 hidden layers

In [ ]:
import models.bert_learned_pooler as bert_learned_pooler

dict = {}
for i in (20, 40, 60, 80):
    bert_model = bert_learned_pooler.create_learned_pooler(i)
    bert_model.load_weights(
        f"./models/learned_pooler_epochs_{i}/training_checkpoints/ckpt_0001.ckpt"
    )
    w = bert_model.weights[-4].numpy()
    top_12 = np.sort(w.argsort()[-12:][::-1])
    dict[i] = repr(top_12)
    del bert_model

print(dict)


### evaluate specific model using squadv2 scripts

In [2]:
from models.model_scorer import generate_scoring_dict
import models.bert_learned_pooler as bert_learned_pooler

i = 1
bert_model = bert_learned_pooler.create_learned_pooler(i)
bert_model.load_weights(f"./models/learned_pooler_epochs_{i}/training_checkpoints/ckpt_0001.ckpt")
w = bert_model.weights[-4].numpy()
print([f"{i:w[i]}" for i in len(w)])
top_12 = np.sort(w.argsort()[-12:][::-1])
print(top_12)


loading the base model from checkpoint 1...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

[-0.7286164  -0.68705434 -0.7754198  -0.7786671  -0.6581254  -0.84351796
 -0.7038068  -0.78308666 -0.69364554 -0.7104143  -0.74511576 -0.67180556
 -0.5480674  -0.5075096  -0.6255021  -0.65797615 -0.6923896  -0.8698429
 -0.75600356 -0.25995982  0.2591863   0.38232687  0.6862729   0.9188513
  1.3424544 ]
[ 4 11 12 13 14 15 19 20 21 22 23 24]


In [12]:
exp_w = [np.exp(i) for i in w]
print(exp_w / sum(exp_w))
print(sum(exp_w[-5:]) / sum(exp_w))


[0.02270178 0.0236652  0.02166374 0.02159351 0.02435981 0.02023759
 0.02327205 0.02149829 0.02350973 0.02311879 0.02233029 0.02402883
 0.02719389 0.02831949 0.02516761 0.02436344 0.02353927 0.01971179
 0.02208848 0.03627396 0.06096172 0.06895035 0.09344124 0.11790877
 0.18010037]
0.5213624548959566


In [ ]:
w = bert_model.weights[-4].numpy()
t = bert_model.weights[-3].numpy()
layer_to_weight = dict(zip(range(len(w)), w))
layer_to_weight


Get the top 12 absolute weights from layer_to_weight

In [ ]:
top_12 = abs(w).argsort()[-12:][::-1]
top_12
